In [1]:
# Необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor

from sklearn.metrics import explained_variance_score as evs
from sklearn.metrics import mean_absolute_error as mae

import hyperopt as hpr
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузка датасета
dataset = pd.read_csv('msk_dataset.csv')
dataset.head()

,id,price,time,metro,address,room_quantity,new_old,house_type,floor_number,floor_quantity,space,kitchen_space,living_space,lang,lat
0,222604784,22498670,2018-05-11 06:36:39,Авиамоторная,Душинская 16,3,Новостройка,NaN,20.0,21.0,135.62,20.02,76.57,37.704754,55.745595
1,241966436,22400000,2018-04-24 08:53:11,Авиамоторная,Авиамоторная 4К1,3,Вторичка,NaN,9.0,17.0,89.00,14.50,55.00,37.713503,55.759603
2,223505840,22359550,2018-05-07 04:32:20,Авиамоторная,Душинская 16,2,Новостройка,NaN,21.0,21.0,90.18,16.26,46.41,37.704754,55.745595
3,187233868,22243093,2018-05-10 16:54:03,Авиамоторная,"ул. Душинская, вл. 16",3,Новостройка,Монолитный,4.0,21.0,136.20,18.70,80.10,37.704805,55.745714
4,246168959,21677040,2018-05-11 06:36:31,Авиамоторная,Душинская 16,3,Новостройка,NaN,9.0,21.0,135.00,21.63,73.69,37.704754,55.745595


In [3]:
# Избавление от ошибок
dataset = dataset[pd.notnull(dataset.address) & pd.notnull(dataset.price) & pd.notnull(dataset.room_quantity)&pd.notnull(dataset.lang)&pd.notnull(dataset.lat)&pd.notnull(dataset.metro)]
print('Длина датасета : ', len(dataset))

Длина датасета :  122107


In [4]:
# Линии метро
red = ['Библиотека имени Ленина', 'Бульвар Рокоссовского', 'Воробьевы горы', 'Комсомольская', 'Красносельская', 'Красные ворота', 'Кропоткинская', 'Лубянка', 'Охотный ряд', 'Парк Культуры', 'Преображенская площадь', 'Проспект Вернадского', 'Румянцево', 'Саларьево', 'Сокольники', 'Спортивная', 'Тропарёво', 'Университет', 'Фрунзенская', 'Черкизовская', 'Чистые пруды', 'Юго-Западная']
green = ['Автозаводская', 'Алма-Атинская', 'Аэропорт', 'Белорусская', 'Водный стадион', 'Войковская', 'Динамо', 'Домодедовская', 'Кантемировская', 'Каширская', 'Коломенская', 'Красногвардейская', 'Маяковская', 'Новокузнецкая', 'Орехово', 'Павелецкая', 'Речной вокзал', 'Сокол', 'Тверская', 'Театральная', 'Технопарк', 'Ховрино', 'Царицыно']
blue = ['Арбатская', 'Бауманская', 'Волоколамская', 'Измайловская', 'Киевская', 'Крылатское', 'Кунцевская', 'Курская', 'Митино', 'Молодежная', 'Мякинино', 'Парк Победы', 'Партизанская', 'Первомайская', 'Площадь Революции', 'Пятницкое шоссе', 'Семеновская', 'Славянский бульвар', 'Смоленская', 'Строгино', 'Щелковская', 'Электрозаводская']
light_blue = ['Александровский сад', 'Арбатская', 'Багратионовская', 'Выставочная', 'Киевская', 'Кунцевская', 'Кутузовская','Международная', 'Пионерская', 'Смоленская', 'Студенческая', 'Филевский парк', 'Фили']
circle = ['Белорусская', 'Добрынинская', 'Киевская', 'Комсомольская', 'Краснопресненская', 'Курская', 'Новослободская', 'Октябрьская', 'Павелецкая', 'Парк Культуры', 'Проспект Мира', 'Таганская']
orange = ['Академическая', 'Алексеевская', 'Бабушкинская', 'Беляево', 'Ботанический сад', 'ВДНХ', 'Калужская', 'Китай-город', 'Коньково', 'Ленинский проспект', 'Медведково', 'Новоясеневская', 'Новые Черёмушки', 'Октябрьская','Проспект Мира', 'Профсоюзная', 'Рижская', 'Свиблово', 'Сухаревская', 'Теплый стан', 'Третьяковская', 'Тургеневская', 'Шаболовская', 'Ясенево']
purple = ['Баррикадная', 'Беговая', 'Волгоградский проспект', 'Выхино', 'Жулебино', 'Китай-город', 'Котельники', 'Кузнецкий мост', 'Кузьминки', 'Лермонтовский проспект', 'Октябрьское поле', 'Планерная', 'Полежаевская', 'Пролетарская', 'Пушкинская', 'Рязанский проспект', 'Спартак', 'Сходненская', 'Таганская', 'Текстильщики', 'Тушинская', 'Улица 1905 года', 'Щукинская']
yellow = ['Авиамоторная', 'Ломоносовский проспект', 'Марксистская', 'Минская', 'Новогиреево', 'Новокосино', 'Парк Победы', 'Перово', 'Петровский парк', 'Площадь Ильича', 'Раменки', 'Третьяковская', 'Хорошёвская', 'ЦСКА', 'Шелепиха', 'Шоссе Энтузиастов']
grey = ['Алтуфьево', 'Аннино', 'Бибирево', 'Боровицкая', 'Бульвар Дмитрия Донского', 'Владыкино', 'Дмитровская', 'Менделеевская', 'Нагатинская', 'Нагорная', 'Нахимовский Проспект', 'Отрадное', 'Петровско-Разумовская', 'Полянка', 'Пражская', 'Савеловская', 'Севастопольская', 'Серпуховская', 'Тимирязевская', 'Тульская', 'Улица академика Янгеля', 'Цветной бульвар', 'Чертановская', 'Чеховская', 'Южная']
light_green = ['Борисово', 'Братиславская', 'Бутырская', 'Верхние Лихоборы', 'Волжская', 'Достоевская', 'Дубровка', 'Зябликово', 'Кожуховская', 'Крестьянская застава', 'Люблино', 'Марьина роща', 'Марьино', 'Окружная', 'Петровско-Разумовская', 'Печатники', 'Римская', 'Селигерская', 'Сретенский бульвар', 'Трубная', 'Фонвизинская', 'Чкаловская', 'Шипиловская', 'Каховская', 'Варшавская', 'Каховская', 'Каширская']
light_purple = ['Битцевский парк', 'Бульвар адмирала Ушакова', 'Бунинская Аллея', 'Лесопарковая', 'Улица Горчакова', 'Улица Скобелевская', 'Улица Старокачаловская']
monorels = ['Выставочный центр', 'Телецентр', 'Тимирязевская', 'Улица Академика Королёва', 'Улица Милашенкова', 'Улица Сергея Эйзенштейна']
msk_circle = ['Автозаводская', 'Андроновка', 'Балтийская', 'Белокаменная', 'Ботанический сад', 'Бульвар Рокоссовского', 'Верхние Котлы', 'Владыкино', 'Дубровка', 'ЗИЛ', 'Зорге', 'Измайлово', 'Коптево', 'Крымская', 'Кутузовская','Лихоборы', 'Локомотив', 'Лужники', 'Нижегородская', 'Новохохловская', 'Окружная', 'Панфиловская', 'Площадь Гагарина', 'Ростокино', 'Соколиная Гора', 'Стрешнево', 'Угрешская', 'Хорошёво', 'Шелепиха','Шоссе Энтузиастов']
big_circle = ['Деловой центр', 'Петровский парк', 'Хорошёвская', 'ЦСКА', 'Шелепиха']
# Станции за МКАДом
mkad = ['Бульвар адмирала Ушакова', 'Бульвар Дмитрия Донского', 'Бунинская Аллея', 'Боровское шоссе', 'Волоколамская', 'Выхино', 'Жулебино', 'Котельники', 'Кропоткинская', 'Кунцевская', 'Кунцевская', 'Кунцевская', 'Лермонтовский проспект', 'Лухмановская', 'Митино', 'Молодежная', 'Мякинино', 'Новокосино', 'Некрасовка', 'Новопеределкино', 'Пятницкое шоссе', 'Румянцево', 'Рассказовка', 'Саларьево', 'Солнцево', 'Улица Горчакова', 'Улица Скобелевская', 'Улица Старокачаловская', 'Улица Дмитриевского']
# Станции внутри кольцевой линии
center = ['Белорусская', 'Добрынинская', 'Киевская', 'Комсомольская', 'Краснопресненская', 'Курская', 'Новослободская', 'Октябрьская', 'Павелецкая', 'Парк Культуры', 'Проспект Мира', 'Таганская', 'Баррикадная', 'Серпуховская', 'Чкаловская', 'Менделеевская', 'Пушкинская', 'Кузнецкий мост', 'Китай-город', 'Маяковская', 'Тверская', 'Театральная', 'Новокузнецкая', 'Цветной бульвар', 'Чеховская', 'Боровицкая', 'Полянка', 'Трубная', 'Сретенский бульвар', 'Сухаревская', 'Тургеневская', 'Третьяковская', 'Красные ворота', 'Чистые пруды', 'Лубянка', 'Охотный ряд', 'Библиотека имени Ленина', 'Кропоткинская', 'Площадь Революции', 'Александровский сад', 'Арбатская', 'Смоленская']

In [5]:
# Обработка значений станций метро и генерация переменных
metro_editor = lambda x: str(x).split(',')[0] if pd.notnull(x)&(str(x).find(',') != -1) else x
dataset.metro = dataset.metro.apply(metro_editor)
liner = lambda x: 'Сокольническая' if x in red else 'Замоскворецкая' if x in green else 'Арбатско-Покр' if x in blue else 'Филевская' if x in light_blue else 'Кольцевая' if x in circle else 'Калужско-Риж' if x in orange else 'Таганско-Красн' if x in purple else 'Калининская' if x in yellow else 'Серпух-Тимиряз' if x in grey else 'Любл-Дмитров' if x in light_green else 'Бутовская' if x in light_purple else 'Монорельс' if x in monorels else 'МЦК' if x in msk_circle else 'БКЛ' if x in big_circle else np.nan
is_in_mkad = lambda x: 0 if (x in mkad) | (pd.isnull(x)) else 1
is_in_center = lambda x: 1 if (x in center) else 0
dataset['line'] = dataset['metro'].apply(liner)
dataset['in_mkad'] = dataset['metro'].apply(is_in_mkad)
dataset['in_center'] = dataset['metro'].apply(is_in_center)

In [6]:
# Избавление от технических переменных и генерация бинарных переменных из качественных (dummies)
dataset = dataset.drop(['id','time','address'], axis = 1)
dataset = pd.get_dummies(dataset)
dataset.head()

,price,floor_number,floor_quantity,space,kitchen_space,living_space,lang,lat,in_mkad,in_center,...,line_Замоскворецкая,line_Калининская,line_Калужско-Риж,line_Кольцевая,line_Любл-Дмитров,line_МЦК,line_Серпух-Тимиряз,line_Сокольническая,line_Таганско-Красн,line_Филевская
0,22498670,20.0,21.0,135.62,20.02,76.57,37.704754,55.745595,1,0,...,0,1,0,0,0,0,0,0,0,0
1,22400000,9.0,17.0,89.00,14.50,55.00,37.713503,55.759603,1,0,...,0,1,0,0,0,0,0,0,0,0
2,22359550,21.0,21.0,90.18,16.26,46.41,37.704754,55.745595,1,0,...,0,1,0,0,0,0,0,0,0,0
3,22243093,4.0,21.0,136.20,18.70,80.10,37.704805,55.745714,1,0,...,0,1,0,0,0,0,0,0,0,0
4,21677040,9.0,21.0,135.00,21.63,73.69,37.704754,55.745595,1,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
# Разделение датасета на тестовую и тренировочную выборки, заполнение пропущенных значений
X = dataset.drop(['price'], axis = 1)
y = dataset.price
sum(pd.isnull(X.floor_number))
for column in ['floor_number', 'floor_quantity', 'space', 'kitchen_space','living_space']:
    mean = np.mean(X[column])
    nan_remover = lambda x: x if pd.notnull(x) else mean
    X[column] = X[column].apply(nan_remover)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

### Первичное постороение моделей

In [12]:
results = dict()

In [13]:
# Случайный лес
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train, y_train)
results['Random Forest'] = [np.round(evs(y_test,rf.predict(X_test)),2), np.round(mae(y_test,rf.predict(X_test)),2)]

In [14]:
# Градиентный бустинг
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)
results['Gradient Boosting'] = [np.round(evs(y_test,gb.predict(X_test)),2), np.round(mae(y_test,gb.predict(X_test)),2)]

In [15]:
# Стохастический градиентный спуск
sgd = SGDRegressor(max_iter = 10, penalty = 'elasticnet')
sgd.fit(X_train, y_train)
results['SGD'] = [np.round(evs(y_test,sgd.predict(X_test)),1), np.round(mae(y_test,sgd.predict(X_test)),1)]

In [16]:
# Адаптивный бустинг
ab = AdaBoostRegressor(n_estimators = 100)
ab.fit(X_train, y_train)
results['Ada Boost'] = [np.round(evs(y_test,ab.predict(X_test)),2), np.round(mae(y_test,ab.predict(X_test)),2)]

In [17]:
df = pd.DataFrame(results, index = ['Explained variance', 'Mean absolute error'])
df

,Ada Boost,Gradient Boosting,Random Forest,SGD
Explained variance,0.65,0.81,0.91,-3.893933e+08
Mean absolute error,9688213.50,4999609.34,2526053.03,1.393116e+12


### Подбор параметров с помощью hyperopt

In [8]:
variable = [ x for x in X_train.columns ]

def hyperopt_train_test( params ):
    try: dell_features = [ x for x in variable if params[x] == 0 ]
    except: return 0.1
    
    try: scores = cross_validation.cross_val_score( model, X_train.drop( dell_features, axis = 1 ), y_train, cv = 3 )
    except: return 0.1
    
    return scores.mean()

def f_rfc( params ):
    acc = hyperopt_train_test( params )
    return { 'loss': -acc, 'status': hpr.STATUS_OK }

model = RandomForestRegressor(n_estimators = 100)
    
space4dt = { x: hpr.hp.choice( x , [0, 1]) for x in variable }
trials = hpr.Trials()

# Находим параметры
best = hpr.fmin( f_rfc, space4dt, algo = hpr.tpe.suggest, max_evals = 500, trials = trials )
dell_features0 = [ x for x in variable if best[x] == 0 ]

In [9]:
X_train.drop(dell_features0, axis = 1).columns

Index(['floor_quantity', 'space', 'lang', 'metro_Автозаводская',
       'metro_Академическая', 'metro_Алма-Атинская', 'metro_Андроновка',
       'metro_Арбатская', 'metro_Аэропорт', 'metro_Бабушкинская',
       ...
       'house_type_Деревянный', 'house_type_Монолитный',
       'house_type_Панельный', 'line_БКЛ', 'line_Замоскворецкая',
       'line_Калужско-Риж', 'line_Любл-Дмитров', 'line_Сокольническая',
       'line_Таганско-Красн', 'line_Филевская'],
      dtype='object', length=132)

In [25]:
results = dict()
# Случайный лес
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train.drop(dell_features0, axis = 1), y_train)
results['Random Forest'] = [np.round(evs(y_test,rf.predict(X_test.drop(dell_features0, axis = 1))),2), np.round(mae(y_test,rf.predict(X_test.drop(dell_features0, axis = 1))),2)]
df = pd.DataFrame(results, index = ['Explained variance', 'Mean absolute error'])
df

,Random Forest
Explained variance,0.91
Mean absolute error,2325864.00


In [27]:
def hyperopt_train_test(params):
    X_ = X_train.drop( dell_features0 , axis = 1 ).values[:]
    if 'normalize' in params:
        if params['normalize'] == 1:
            X_ = normalize(X_)
            del params['normalize']

    if 'scale' in params:
        if params['scale'] == 1:
            X_ = scale(X_)
            del params['scale']
    model = RandomForestRegressor(**params)
    model.fit(X_,y_train.values)    
    return cross_val_score(model, X_, y_train.values).mean()

space4rf = {
    'max_depth': hp.choice('max_depth', [5,10,None]),
    'max_features': hp.choice('max_features', [5,10,None]),
    'n_estimators': hp.choice('n_estimators', range(1,200,10)),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1,5)),
    'max_leaf_nodes': hp.choice('max_leaf_nodes', range(2,20,4))
}

best = 0
def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc > best:
        best = acc
#         print('new best: ', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4rf, algo=tpe.suggest, max_evals=300, trials=trials)
print('best:')
print(best)

best:
{'max_depth': None, 'max_features': None, 'max_leaf_nodes': 16, 'min_samples_leaf': 2, 'n_estimators': 150}


In [24]:
results = dict()
# Случайный лес
rf = RandomForestRegressor(n_estimators = 100, max_features = None)
rf.fit(X_train.drop(dell_features0, axis = 1), y_train)
results['Random Forest'] = [np.round(evs(y_test,rf.predict(X_test.drop(dell_features0, axis = 1))),2), np.round(mae(y_test,rf.predict(X_test.drop(dell_features0, axis = 1))),2)]
df = pd.DataFrame(results, index = ['Explained variance', 'Mean absolute error'])
df

,Random Forest
Explained variance,0.94
Mean absolute error,1325864.00
